In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import date
import time
import psycopg2
import json
import requests

In [2]:
def dbconn():
    conn = psycopg2.connect(host="ec2-23-21-207-93.compute-1.amazonaws.com", 
                            dbname="d3oubpekvnbupv", 
                            user="grxhirqndvyqvv", 
                            password="6f1afaafe16d245c70666bdb8c831aa876e62b380a1544f5dc832c51f27cece6", 
                            port="5432")
    return conn

In [3]:
# DB - 오늘날짜의 지하철 이슈 찾아보기
def todaysubaccDB():
    today = date.today().isoformat() + '%'
    
    cur = dbconn().cursor()
    cur.execute(f"SELECT * FROM subaccdata WHERE acctime LIKE '{today}' order by acctime DESC")
    result_all = cur.fetchall()
    sbstr=""
    
    for i in result_all:
        for j in i:
            sbstr =  sbstr + j + "\n"
    
    sbstr = sbstr[:-1]
    
    return sbstr

In [4]:
# SELENIUM - 오늘 날짜의 지하철 이슈 찾아보기
def todaysubaccCROW():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("headless")
    driver = webdriver.Chrome('/chromedriver.exe', options=chrome_options)
    time.sleep(5)
    driver.get("https://safecity.seoul.go.kr/acdnt/sbwyIndex.do")
    time.sleep(0.5)
    parentElement = driver.find_elements(By.XPATH, '//*[@id="dv_as_timeline"]/li')
    subli=[]
    for i in parentElement:
        i.click()
        time.sleep(0.05)
        a = i.text
        subli.append(a)
        i.click()
    driver.quit()
    return subli

In [7]:
a = todaysubaccCROW()

C:\Users\h\AppData\Local\Temp\ipykernel_23932\2624937414.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/chromedriver.exe', options=chrome_options)


In [8]:
a

['지하철사고\n2022-11-25 06:05\n서울교통공사에서 알려드립니다. 금일 08시부터 4호선에서「전국장애인차별철폐연대」의 ‘장애인 권리예산 확보’를 위한 출근길 지하철 타기 선전전이 예정되어 있습니다. 이로 인해 4호선 해당구간 열차운행이 상당시간 지연될 수 있으니 이점 참고하여 열차를 이용해 주시기 바랍니다.']

In [5]:
def kakaoapi(message):
    with open("kakao_code.json","r") as fp:
        ts = json.load(fp)
        
    url="https://kapi.kakao.com/v2/api/talk/memo/default/send"
    headers={"Authorization" : "Bearer " + ts["access_token"]}
    data={
        "template_object": json.dumps({
            "object_type":"text",
            "text":message,
            "link":{
                "web_url":"www.naver.com"
            }
        })
    }
    response = requests.post(url, headers=headers, data=data)
    if response.status_code==200:
        print("카톡알림 완료")
    else:
        print("카톡알림 fail :", response.status_code)

In [14]:
with open("kakao_code.json","r") as fp:
    ts = json.load(fp)
        
url="https://kapi.kakao.com/v2/api/talk/memo/default/send"
headers={"Authorization" : "Bearer " + ts["access_token"]}
data={
    "template_object": json.dumps({
        "object_type":"text",
        "text":"왜안댐~~",
        "link":{
            "web_url":"www.naver.com"
        }
    })
}
response = requests.post(url, headers=headers, data=data)

In [6]:
# main코드
def main():
    dbselect = todaysubaccDB()
    crwaling = todaysubaccCROW()
    
    # 크롤링한 데이터 나누기 - DB에서 가져온 것과 비교하기 위해
    dbstr = ""
    for i in crwaling:
        i = i[6:]
        dbstr = dbstr + i + '\n'
    dbstr = dbstr[:-1]
    
    # db데이터와 크롤링한 데이터가 다를때
    if (dbstr!=dbselect):
        
        # 1. 속보가 없는건지 확인한다
        if (len(crwaling[0]) == 12):
            print("속보없음")
        
        # 2. 속보가 있으면 db에 넣어주고 카톡 알림
        else:
            conn = dbconn()
            cur = conn.cursor()
            
            for i in crwaling:
                # 크롤링한 데이터 정규화
                subdb = i.split('\n')
                subdb.pop(0)
                cur.execute("INSERT into subaccdata(acctime, content) VALUES (%s, %s) ON CONFLICT (acctime, content) DO NOTHING", subdb)
            conn.commit()
            
            # 카카오 메시지 보내기 호출
            kakaoapi(crwaling[0])

In [44]:
main()

C:\Users\h\AppData\Local\Temp\ipykernel_23224\2624937414.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/chromedriver.exe', options=chrome_options)


In [17]:
conn = dbconn()
cur = conn.cursor()
conn.commit()

In [24]:
a = todaysubaccDB()

In [25]:
a

''

In [ ]:
2022-11-25 06:05

In [23]:
cur.execute(f"delete from subaccdata where acctime = '2022-11-25 06:05'")
conn.commit()